In [274]:
import requests
import pandas as pd
from config import fund_dict
import xml.etree.ElementTree as ET

## Scrapping Data

In [252]:
_BASE_URL_SEC_API = "https://data.sec.gov"
_BASE_URL_XBRL = f"{_BASE_URL_SEC_API}/api/xbrl"

# SEC API endpoints as documented here:
# https://www.sec.gov/edgar/sec-api-documentation
BASE_URL_SUBMISSIONS = f"{_BASE_URL_SEC_API}/submissions"
cik = '0001535392'
accessionNumber='000117266122002482'
BASE_URL_FORM_TABLE = f"{_BASE_URL_SEC_API}/Archives/edgar/data/{cik}/{accessionNumber}/xslForm13F_X01/infotable.xml"
#https://www.sec.gov/edgar/browse/?CIK=1535392
#accession numner: 000117266122002482
print('https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482/xslForm13F_X01/infotable.xml')

https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482/xslForm13F_X01/infotable.xml


In [253]:
#https://www.sec.gov/os/webmaster-faq#privacy-enhanced-message
headers = {'User-Agent':
           'StockFries stockfries@gmail.com'}
api_endpoint = f"{BASE_URL_SUBMISSIONS}/CIK{cik}.json"
print(api_endpoint)
r = requests.get(url=api_endpoint, headers=headers)
print(r.json()['filings'].keys())
print(r.json()['filings']['recent'].keys())
#print(r.json()['filings']['recent']['form'])
print(r.json()['filings']['recent']['filingDate'][0])
print(r.json()['filings']['recent']['reportDate'][0])
r.json()['filings']['recent']['accessionNumber'][0]
accessionNumber=r.json()['filings']['recent']['accessionNumber'][0].replace('-','')
print(accessionNumber)

https://data.sec.gov/submissions/CIK0001535392.json
dict_keys(['recent', 'files'])
dict_keys(['accessionNumber', 'filingDate', 'reportDate', 'acceptanceDateTime', 'act', 'form', 'fileNumber', 'filmNumber', 'items', 'size', 'isXBRL', 'isInlineXBRL', 'primaryDocument', 'primaryDocDescription'])
2022-11-14
2022-09-30
000117266122002482


In [286]:
cik_no_pad='1535392'
BASE_URL_FORM_TABLE = f"https://www.sec.gov/Archives/edgar/data/{cik_no_pad}/{accessionNumber}/xslForm13F_X01/infotable.xml"
print(BASE_URL_FORM_TABLE)
BASE_URL_FORM_TABLE = f"https://www.sec.gov/Archives/edgar/data/{cik_no_pad}/{accessionNumber}/infotable.xml"

print(BASE_URL_FORM_TABLE)
headers = {'User-Agent':
           'StockFries stockfries@gmail.com'}
api_endpoint = BASE_URL_FORM_TABLE
print(api_endpoint)
r = requests.get(url=api_endpoint, headers=headers)
#r.json()
#xml_updated = ET.fromstring(str(r.content).replace('<shrsOrPrnAmt>','').replace('</shrsOrPrnAmt>',''))
#print(r.content)
xml_updated = ET.fromstring(r.content)
#parser = ET.XMLParser(encoding="utf-8")
#tree = ET.parse(r.content, parser=parser)

#print( ET.fromstring(str(r.content)))
df = pd.read_xml(r.content)


df.tail()

https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482/xslForm13F_X01/infotable.xml
https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482/infotable.xml
https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482/infotable.xml


,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority
368,XPO LOGISTICS INC,COM,983793100,14246,NaN,SOLE,NaN
369,YOTTA ACQUISITION CORPORATIO,RIGHT 03/15/2027,98741Y129,117,NaN,SOLE,NaN
370,YOTTA ACQUISITION CORPORATIO,*W EXP 03/15/202,98741Y111,9396,NaN,SOLE,NaN
371,ZIMMER ENERGY TRANSITION ACQ,COM CL A,989570106,149,NaN,SOLE,NaN
372,Z-WORK ACQUISITION CORP,*W EXP 01/04/202,98880C110,322,NaN,SOLE,NaN


In [257]:
def create_holdings_df(cik_='1535392',fund_name_='Mangrove Partners'):
    #Pull Filing accessionNumbers
    headers_ = {'User-Agent':
           'StockFries stockfries@gmail.com'}
    cik_padded_ = '0001535392'
    url_submissions = f'https://data.sec.gov/submissions/CIK{cik_padded_}.json'
    #print(url_submissions)
    r_ = requests.get(url=url_submissions, headers=headers)
    
    #Pull Latest 13F-HR Filing
    new_13F_level='None'
    old_13F_level='None'
    for index, filing_form in enumerate(r_.json()['filings']['recent']['form']):
        if (filing_form=='13F-HR') & (new_13F_level=='None'):
            new_13F_level = index
            continue
        elif ((filing_form=='13F-HR') & (new_13F_level!='None') & (old_13F_level=='None')):
            old_13F_level = index
            break
        #elif (new_13F_level!='None') & (new_13F_level!='None'):
        #    break
        else:
            continue
    #print('new_13F_level', 'old_13F_level')
    #print(new_13F_level, old_13F_level)
            
    new_accessionNumber=r_.json()['filings']['recent']['accessionNumber'][new_13F_level].replace('-','')
    new_filingDate = r_.json()['filings']['recent']['filingDate'][new_13F_level]
    new_reportDate = r_.json()['filings']['recent']['reportDate'][new_13F_level]
    new_form = r_.json()['filings']['recent']['form'][new_13F_level]
    old_accessionNumber=r_.json()['filings']['recent']['accessionNumber'][old_13F_level].replace('-','')
    old_filingDate = r_.json()['filings']['recent']['filingDate'][old_13F_level]
    old_reportDate = r_.json()['filings']['recent']['reportDate'][old_13F_level]
    old_form = r_.json()['filings']['recent']['form'][old_13F_level]
    
    url_filling_new = f"https://www.sec.gov/Archives/edgar/data/{cik_}/{new_accessionNumber}/infotable.xml"
    #print(url_filling_new)
    r_filing_new = requests.get(url=url_filling_new, headers=headers)
    df_new = pd.read_xml(r_filing_new.content)
    df_new['cik']=cik_
    df_new['filingDate']= new_filingDate
    df_new['reportDate']= new_reportDate
    
    #Pull Previous Filing
    url_filling_old = f"https://www.sec.gov/Archives/edgar/data/{cik_}/{old_accessionNumber}/infotable.xml"
    #print(url_filling_old)
    r_filing_old = requests.get(url=url_filling_old, headers=headers)
    df_old = pd.read_xml(r_filing_old.content)
    df_old['cik']=cik_
    df_old['filingDate']= old_filingDate
    df_old['reportDate']= old_reportDate
    df_final = pd.concat([df_new, df_old])
    df_final['fund_name'] = fund_name_
    
    return df_final

In [197]:
df = create_holdings_df()

df.tail()

https://data.sec.gov/submissions/CIK0001535392.json
https://www.sec.gov/Archives/edgar/data/1535392/000117266122002482/infotable.xml
https://www.sec.gov/Archives/edgar/data/1535392/000117266122001938/infotable.xml


,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,cik,filingDate,reportDate,fund_name
515,WINVEST ACQUISITION CORP,*W EXP 08/09/202,97655B117,4843,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners
516,XPAC ACQUISITION CORP,CLASS A ORD,G9831X106,4880,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners
517,YOTTA ACQUISITION CORPORATIO,UNIT 03/15/2027,98741Y202,4487,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners
518,YOTTA ACQUISITION CORPORATIO,RIGHT 03/15/2027,98741Y129,120,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners
519,ZENDESK INC,COM,98936J101,11701,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners


## Process Data

In [258]:
#print(fund_dict.keys())
df_all = pd.DataFrame()
for fund in fund_dict:
    try:
        fund_name = fund
        cik = fund_dict[fund].split('CIK=')[1].split('&')[0]
        #print(cik)
        df_fund = create_holdings_df()
        df_all = pd.concat([df_all, df_fund]).reset_index(drop=True)
    except:
        continue
print(df_all.columns)
df_all.tail()

Index(['nameOfIssuer', 'titleOfClass', 'cusip', 'value', 'shrsOrPrnAmt',
       'investmentDiscretion', 'votingAuthority', 'cik', 'filingDate',
       'reportDate', 'fund_name'],
      dtype='object')


,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,cik,filingDate,reportDate,fund_name
50003,WINVEST ACQUISITION CORP,*W EXP 08/09/202,97655B117,4843,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners
50004,XPAC ACQUISITION CORP,CLASS A ORD,G9831X106,4880,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners
50005,YOTTA ACQUISITION CORPORATIO,UNIT 03/15/2027,98741Y202,4487,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners
50006,YOTTA ACQUISITION CORPORATIO,RIGHT 03/15/2027,98741Y129,120,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners
50007,ZENDESK INC,COM,98936J101,11701,NaN,SOLE,NaN,1535392,2022-08-15,2022-06-30,Mangrove Partners


In [225]:
print('investmentDiscretion', set(df.investmentDiscretion.values))
#print('shrsOrPrnAmt', set(df.shrsOrPrnAmt.values))
periods = list(set(df.reportDate.values))
print(periods[0])
df_all.reset_index().tail()
df_all[df_all.reportDate == periods[0]].reset_index().tail()
df_current = df_all[df_all.reportDate == max(periods)].reset_index(drop=True)
df_previous = df_all[df_all.reportDate == min(periods)].reset_index(drop=True)

investmentDiscretion {'SOLE'}
2022-06-30


In [232]:
df_current_g = df_current.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)
df_previous_g = df_previous.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)
df_previous_g.tail()

/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_21277/3470449611.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_current_g = df_current.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)
/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_21277/3470449611.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_previous_g = df_previous.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)


,,value,shrsOrPrnAmt,votingAuthority
nameOfIssuer,cusip,,,
BREEZE HOLDINGS ACQUISITN CO,106762123,1568,0.0,0.0
PACIFICO ACQUISITION CORP,69512X111,1232,0.0,0.0
INTERNATIONAL MEDIA ACQUISIT,459867123,1120,0.0,0.0
MOUNTAIN CREST ACQUSITN CRP,62403K116,952,0.0,0.0
PARABELLUM ACQUISITION CORP,69901P117,0,0.0,0.0


In [259]:
def process_scraped_data(df_all_):
    periods = list(set(df_all_.reportDate.values))
    #print(periods[0])
    df_all_.reset_index().tail()
    df_all_[df_all.reportDate == periods[0]].reset_index().tail()
    df_current = df_all[df_all.reportDate == max(periods)].reset_index(drop=True)
    df_previous = df_all[df_all.reportDate == min(periods)].reset_index(drop=True)
    df_current_g = df_current.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)
    df_previous_g = df_previous.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)
    df_diff = df_current_g - df_previous_g
    df_heavy_ = df_current_g
    df_hot_ = df_diff[df_diff.value > 0]
    df_cold_ = df_diff[df_diff.value < 0]
    return df_heavy_, df_hot_, df_cold_
    

## Full Flow

In [260]:
df_all = pd.DataFrame()
for fund in fund_dict:
    try:
        fund_name = fund
        cik = fund_dict[fund].split('CIK=')[1].split('&')[0]
        df_fund = create_holdings_df()
        df_all = pd.concat([df_all, df_fund]).reset_index(drop=True)
        print(fund_name, cik)
    except:
        continue
df_heavy, df_hot, df_cold = process_scraped_data(df_all)

ABRAMS CAPITAL MANAGEMENT, L.P. 1358706
AKRE CAPITAL MANAGEMENT 1112520
ANSON FUNDS 1491072
APPALOOSA LP 1656456
ARLINGTON VALUE CAPITAL, LLC 1568820
ATLANTIC INVESTMENT MANAGEMENT, INC. 1063296
BAKER BROS. ADVISORS LP 1263508
BOGLE INVESTMENT MANAGEMENT L P 0001131667
BLOOMBERGSEN U.S. FUND LP 1641744
CAPITAL FUND MANAGEMENT 1323645
CANYON CAPITAL ADVISORS LLC 0001074034
COATUE MANAGEMENT LLC 1135730
CONTRARIAN CAPITAL MANAGEMENT 1050417
CORVEX MANAGEMENT LP 1535472
DALAL STREET, LLC 1549575
DORSAL CAPITAL MANAGEMENT 1547007
ELLIOTT MANAGEMENT CORP 1048445
EMINENCE CAPITAL, LP 1107310
GATES CAPITAL MANAGEMENT, INC 1312908
GLENVIEW CAPITAL MANAGEMENT, LLC 1138995
HAWK RIDGE CAPITAL MANAGEMENT 1609074
HARVEST CAPITAL STRATEGIES 1226355
HIGHBRIDGE CAPITAL MANAGEMENT LLC 0000919185
HOUND PARTNERS, LLC 1353316
ICAHN CARL C 921669
III Capital Management  0001783139
JANA PARTNERS LLC 1159159
JOHO CAPITAL LLC 1106500
LONE PINE CAPITAL LLC 1061165
LONG POND CAPITAL, LP 1499066
MASTERS CAPITAL 

/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_21277/2799315690.py:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_current_g = df_current.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)
/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_21277/2799315690.py:9: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_previous_g = df_previous.groupby(['nameOfIssuer','cusip']).sum().sort_values('value', ascending=False)


In [62]:
import requests
from xml.etree import ElementTree
tree = ElementTree.fromstring(r.content)

In [9]:
#cik = validate_cik(cik)
api_endpoint = f"{BASE_URL_SUBMISSIONS}/CIK{cik}.json"
print('https://data.sec.gov/submissions/CIK##########.json')
submissions = self._rate_limited_get(api_endpoint)

filings = submissions["filings"]
paginated_submissions = filings["files"]

# Handle pagination for a large number of requests
if handle_pagination and paginated_submissions:
    to_merge = [filings["recent"]]
    for submission in paginated_submissions:
        filename = submission["name"]
        api_endpoint = f"{BASE_URL_SUBMISSIONS}/{filename}"
        resp = self._rate_limited_get(api_endpoint)
        to_merge.append(resp)

    # Merge all paginated submissions from files key into recent
    # and clear files list.
    filings["recent"] = merge_submission_dicts(to_merge)
    filings["files"] = []


https://data.sec.gov/submissions/CIK##########.json


NameError: name 'self' is not defined